## Imports

In [ ]:
import tensorflow as tf
from keras import datasets, layers, models
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
from time import sleep
import cv2
import pandas as pd

## Making the dataset

In [ ]:
# Capture video from webcam and make dataset
class MakeDataset():
    def __init__(self):
        import cv2
        import os
        
        self.video = cv2.VideoCapture(0)
        self.count = 0
        self.person_name = input("Enter person name: ")
        # self.num_images = int(input("Enter number of images: "))
        self.path = "data/" + self.person_name
        if not os.path.exists(self.path):
            os.makedirs(self.path)
    def  make(self):
        while True:
            _, frame = self.video.read()
            cv2.imshow("frame", frame)
            if cv2.waitKey(1) & 0xFF == ord('s'):
                self.count += 1
                cv2.imwrite(self.path + "/" + self.person_name + '_' + str(self.count) + ".jpg", frame)
                print(f'Image saved: {self.count}', end='\r')
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if self.count == 50:
                break
    def __final__(self):
        self.video.release()
        cv2.destroyAllWindows()

# MakeDataset().make()

## Loading the data

### 1. Listing images in the dataset directory

In [ ]:
# # Devasheesh's faces
# deva_faces = './data/devasheesh/'
# deva_faces = os.listdir(deva_faces)
# deva_faces = [deva_faces[i] for i in range(len(deva_faces)) if deva_faces[i].endswith('.jpg')]

# # Swarnim's faces
# swar_faces = './data/swarnim/'
# swar_faces = os.listdir(swar_faces)
# swar_faces = [swar_faces[i] for i in range(len(swar_faces)) if swar_faces[i].endswith('.jpg')]

# # Negative faces
# neg_faces = './data/negative-faces/'
# neg_faces = os.listdir(neg_faces)
# neg_faces = [neg_faces[i] for i in range(len(neg_faces)) if neg_faces[i].endswith('.jpg')]

# Making a function to list all the images in a folder
def list_images(path):
    image_files = []
    for root, _, files in os.walk(path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_files.append(os.path.join(root, file))
    image_files.sort()
    return image_files

deva_faces = list_images('./data/Devasheesh/')
swar_faces = list_images('./data/Swarnim/')
neg_faces = list_images('./data/negative-faces/')

### 2. Making Train and Test Objects

In [ ]:
# function to extract the face from the image
# using the Haar Cascade Classifier
def extract_frontal_face_harr(image_ndarray, grayscale=True, size=(150, 150)):
    # Load the cascade
    face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
    # Convert into grayscale
    if grayscale:
        image_gray = cv2.cvtColor(image_ndarray, cv2.COLOR_BGR2GRAY)
    else:
        image_gray = image_ndarray
    # Detect faces
    faces_cord = face_cascade.detectMultiScale(image_gray, 1.3, 5)
    # Return the face or None if not found
    if len(faces_cord) == 0:
        return None, None
    # Extract the face
    (x, y, w, h) = faces_cord[0]
    # Resize the image to 150x150
    image_gray_resized = cv2.resize(image_gray[y:y+w, x:x+h], size)
    # Return only the face part of the image
    return image_gray_resized, faces_cord

# using DNN
modelFile = "dnn/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "dnn/deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

def extract_frontal_face(image_ndarray, grayscale=False, size=(150, 150)):
    # resize the image to 300x300 for the DNN model
    h, w = 300, 300
    image_ndarray = cv2.resize(image_ndarray, (h, w))
    # Convert into blob
    blob = cv2.dnn.blobFromImage(image_ndarray, 1.0, (h, w), (104.0, 177.0, 123.0))
    # Convert into grayscale
    if grayscale:
        image_ndarray = cv2.cvtColor(image_ndarray, cv2.COLOR_BGR2GRAY)
    # Detect faces
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            # Get the coordinates of the bounding box
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x2, y2) = box.astype("int")

            # Extract the face ROI (region of interest) from the image
            face = image_ndarray[y:y2, x:x2]
            return cv2.resize(face, size), (x, y, x2, y2)
        else:
            return None, None


In [ ]:
# function to make the image size uniform and make train and test object with labels
class dataframe():
    def __init__(self):
        self.deva_faces = deva_faces
        self.swar_faces = swar_faces
        self.neg_faces = neg_faces
        self.images, self.labels = list(), list()

        self.total_images = len(deva_faces)+len(swar_faces)+len(neg_faces)
        self.labels_list = ['Unknown', 'Devasheesh', 'Swarnim']

        print('Total images: ', self.total_images)

        self.datagen = ImageDataGenerator(
            rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.2, # randomly shift images vertically (fraction of total height)
            shear_range=0.2,    # set range for random shear
            zoom_range=0.2,    # set range for random zoom
            horizontal_flip=True,   # randomly flip images
            fill_mode='nearest' # set mode for filling points outside the input boundaries
        )

    def append_dataframe(self, images_list, label, augment=False, grayscale=False):
        count = 1
        for image_adr in images_list:
            # read the image
            image = cv2.imread(image_adr)
            # extract the face
            face, _ = extract_frontal_face(image, grayscale=grayscale, size=(300, 300))

            print('Count: ', count, ' | Image address: ', image_adr, ' | Label: ',self.labels_list[label], ' | Image shape: ', image.shape)
            if face is not None:
                # print('Face type: ', type(face))
                # print('Image shape: ', image.shape)
                # print('Face shape: ', face.shape)
                # print('Face: ', plt.imshow(face))

                # append the face image to the list
                if augment:
                    # using the image data generator to augment the images
                    i = 0
                    for batch in self.datagen.flow(face.reshape(1, 300, 300, 3), batch_size=1):
                        # append the image to the list
                        self.images.append(batch[0])
                        self.labels.append(label)
                        i += 1
                        if i > 10:
                            break
                    count += 1

                else:
                    self.images.append(face)
                    self.labels.append(label)
                    count += 1
            else:
                print(f'Face not found in {image_adr}')
                continue
    
    def make_dataframe(self):
        self.append_dataframe(self.neg_faces, 0, augment=False)
        self.append_dataframe(self.deva_faces, 1, augment=True)
        self.append_dataframe(self.swar_faces, 2, augment=True)
        self.images = np.array(self.images)
        self.labels = np.array(self.labels)
        print('Images shape: ', self.images.shape)
        print('Labels shape: ', self.labels.shape)
        return self.images/255.0, self.labels, self.labels_list

x = dataframe()
images, labels, labels_list = x.make_dataframe()

In [ ]:
# for x in range(len(images)):
#     try:
#         plt.imshow(images[x])
#         plt.title(labels_list[int(labels[x])])
#         plt.show()
#     except:
#         print(f'Error in {x}')

## Splitting the data into train and test

In [ ]:
# split the data into train and test
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2)

In [ ]:
images.shape

In [ ]:
train_images.shape

In [ ]:
test_images.shape

## Creating The CNN Model

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300,300,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(labels_list), activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=2, 
                    validation_data=(test_images, test_labels))

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)


In [ ]:
model.save('face_recognition_model_ep2.h5')

In [ ]:
model.load_weights('face_recognition_model_ep2.h5')

In [ ]:
def predict(model, labels_list, grayscale=False):
    # initialize the video capture object

    # read the frame from the webcam
    # frame = cv2.imread('data\Devasheesh\Devasheesh_32.jpg')
    while True:
        cap = cv2.VideoCapture(0)
        ret, frame = cap.read()
        frame = cv2.resize(frame, (300, 300))
        # extract the face
        face, (x, y, x2, y2) = extract_frontal_face(
            frame, grayscale=grayscale, size=(300, 300))
        if face is not None:
            # predict the face
            pred = model.predict(face.reshape(1, 300, 300, 3))
            # get the label
            label = labels_list[np.argmax(pred)]
            # draw the rectangle and put the text
            cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # show the frame
        cv2.imshow('Face Recognition', frame)

# call the predict function
predict(model, labels_list, grayscale=False)

# destroy all windows
cv2.destroyAllWindows()


In [1]:
# Test the model - Self independent test

# imports
import cv2
from tensorflow import keras
import numpy as np
# load the model
model = keras.models.load_model(r"C:\models\face_recognition_model_ep2.h5")
# using DNN
modelFile = "dnn/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "dnn/deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)


def extract_frontal_face(image_ndarray, size, grayscale=False):
    # resize the image to 300x300 for the DNN model
    h, w = 300, 300
    image_ndarray = cv2.resize(image_ndarray, (h, w))
    # Convert into blob
    blob = cv2.dnn.blobFromImage(
        image_ndarray, 1.0, (h, w), (104.0, 177.0, 123.0))
    # Convert into grayscale
    if grayscale:
        image_ndarray = cv2.cvtColor(image_ndarray, cv2.COLOR_BGR2GRAY)
    # Detect faces
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            # Get the coordinates of the bounding box
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x2, y2) = box.astype("int")

            # Extract the face ROI (region of interest) from the image
            face = image_ndarray[y:y2, x:x2]
            return cv2.resize(face, size), (x, y, x2, y2)
        else:
            return None, None


def predict_video(model, labels_list):
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        print(frame.shape)
        frame = cv2.resize(frame, (300, 300))
        try:
            # extract the face
            face, (x, y, x2, y2) = extract_frontal_face(
                frame, size=(300, 300))
            if face is not None:
                # predict the face
                pred = model.predict(face.reshape(1, 300, 300, 3))
                # get the label
                label = labels_list[np.argmax(pred)]
                # draw the rectangle and put the text
                cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x, y-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            # show the frame
            cv2.imshow('Face Recognition', frame)
        except:
            print("No face detected")


# call the predict function
predict_video(model, labels_list=['Unknown', 'Devasheesh', 'Swarnim'])


(480, 640, 3)
No face detected
(480, 640, 3)
No face detected
(480, 640, 3)
No face detected
(480, 640, 3)
No face detected
(480, 640, 3)
No face detected
(480, 640, 3)
No face detected
(480, 640, 3)
No face detected
(480, 640, 3)
1/1 [==============================] - 0s 116ms/step
(480, 640, 3)
1/1 [==============================] - 0s 37ms/step
(480, 640, 3)
1/1 [==============================] - 0s 36ms/step
(480, 640, 3)
1/1 [==============================] - 0s 36ms/step
(480, 640, 3)
1/1 [==============================] - 0s 36ms/step
(480, 640, 3)
1/1 [==============================] - 0s 34ms/step
(480, 640, 3)
1/1 [==============================] - 0s 36ms/step
(480, 640, 3)
1/1 [==============================] - 0s 35ms/step
(480, 640, 3)
1/1 [==============================] - 0s 41ms/step
(480, 640, 3)
1/1 [==============================] - 0s 38ms/step
(480, 640, 3)
1/1 [==============================] - 0s 38ms/step
(480, 640, 3)
1/1 [==============================] - 0s 

: 

: 